#### 1. Import libraries

In [60]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


#### 2. Scrap data from Wikipedia page into a DataFrame

In [61]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Districts_of_Paris").text

In [62]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [63]:
# create a list to store neighborhood data
neighborhoodList = []

In [64]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [65]:
# create a new DataFrame from the list
Paris_df = pd.DataFrame({"Neighborhood": neighborhoodList})

Paris_df.head()

,Neighborhood
0,Historical quarters of Paris
1,Revolutionary sections of Paris
2,Batignolles
3,"Belleville, Paris"
4,Bercy


In [66]:
# print the number of rows of the dataframe
Paris_df.shape

(31, 1)

#### 3. Get the geographical coordinates

In [67]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Paris,France'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [68]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in Paris_df["Neighborhood"].tolist() ]

In [69]:
coords

[[48.85717000000005, 2.3414000000000215],
 [48.85717000000005, 2.3414000000000215],
 [48.88333000000006, 2.3166700000000446],
 [48.87018000000006, 2.384230000000059],
 [48.834880000000055, 2.3845900000000597],
 [48.8926125, 2.320326000000004],
 [45.748649930607726, 4.881890076605707],
 [48.84296194242569, 2.325297507865362],
 [48.85094000000004, 2.3756700000000706],
 [48.85780712325982, 2.3238257190499585],
 [48.84931001832194, 2.2857799561429717],
 [48.88501007660573, 2.354129959461189],
 [48.84842000000003, 2.2916400000000294],
 [48.86258644828371, 2.335725791088914],
 [48.843870018321915, 2.2861300438571446],
 [48.88414983634153, 2.3570929845535393],
 [48.84774997056971, 2.3517999888915306],
 [48.86319000000003, 2.342010000000073],
 [48.85875996335633, 2.360870029430404],
 [48.86988084033124, 2.394114768894286],
 [48.88615000000004, 2.337970000000041],
 [48.84313000000003, 2.321290000000033],
 [48.832059959461205, 2.339659919499214],
 [46.54055000000005, 4.540730000000053],
 [48.826

In [70]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
coords_df = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [71]:
# merge the coordinates into the original dataframe
Paris_df['Latitude'] = coords_df ['Latitude']
Paris_df['Longitude'] = coords_df ['Longitude']

In [72]:
# check the neighborhoods and the coordinates
print(Paris_df.shape)
Paris_df

(31, 3)


,Neighborhood,Latitude,Longitude
0,Historical quarters of Paris,48.857170,2.341400
1,Revolutionary sections of Paris,48.857170,2.341400
2,Batignolles,48.883330,2.316670
3,"Belleville, Paris",48.870180,2.384230
4,Bercy,48.834880,2.384590
5,Cité des Fleurs,48.892612,2.320326
6,Cour des miracles,45.748650,4.881890
7,Épinettes,48.842962,2.325298
8,Faubourg Saint-Antoine,48.850940,2.375670
9,Faubourg Saint-Germain,48.857807,2.323826


In [73]:
# save the DataFrame as CSV file
Paris_df.to_csv("Paris_df.csv", index=False)

#### 4. Create a map of Kuala Lumpur with neighborhoods superimposed on top

In [74]:
# get the coordinates of Kuala Lumpur
address = 'Paris, France'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris, France {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris, France 48.8566101, 2.3514992.


In [75]:
# create map of Toronto using latitude and longitude values
map_Paris = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(Paris_df['Latitude'], Paris_df['Longitude'], Paris_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Paris)  
    
map_Paris

In [81]:
# save the map as HTML file
map_Paris.save('map_Paris.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [82]:
# define Foursquare Credentials and Version
CLIENT_ID = 'ZNVSTRHLFNHQ3H25OTVR3HXHPZVWQFNPBSN3AGBVI5DAPGVG' # your Foursquare ID
CLIENT_SECRET = 'NA0KEE1RDJHIYGXTU1NUE3RQDIOQOPGXVMJ4VCHLAARDYKVG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZNVSTRHLFNHQ3H25OTVR3HXHPZVWQFNPBSN3AGBVI5DAPGVG
CLIENT_SECRET:NA0KEE1RDJHIYGXTU1NUE3RQDIOQOPGXVMJ4VCHLAARDYKVG


In [83]:
# get the top 100 venues that are within a radius of 2000 meters

radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(Paris_df['Latitude'], Paris_df['Longitude'], Paris_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [84]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2859, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Historical quarters of Paris,48.85717,2.3414,Chez Nous,48.855823,2.340271,Wine Bar
1,Historical quarters of Paris,48.85717,2.3414,Ze Kitchen Galerie,48.855101,2.341297,Restaurant
2,Historical quarters of Paris,48.85717,2.3414,Place Dauphine,48.856640,2.342362,Plaza
3,Historical quarters of Paris,48.85717,2.3414,Place du Louvre,48.859841,2.340822,Plaza
4,Historical quarters of Paris,48.85717,2.3414,Pont des Arts,48.858565,2.337635,Bridge


In [85]:
# Let's check how many venues were returned for each neighorhood

venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Batignolles,100,100,100,100,100,100
"Belleville, Paris",100,100,100,100,100,100
Bercy,100,100,100,100,100,100
Cité des Fleurs,100,100,100,100,100,100
Cour des miracles,54,54,54,54,54,54
Faubourg Saint-Antoine,100,100,100,100,100,100
Faubourg Saint-Germain,100,100,100,100,100,100
Front de Seine,100,100,100,100,100,100
Goutte d'Or,100,100,100,100,100,100


In [86]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))


There are 206 uniques categories.


In [87]:
venues_df['VenueCategory'].unique()[:206]

array(['Wine Bar', 'Restaurant', 'Plaza', 'Bridge', 'Cocktail Bar',
       'Pedestrian Plaza', 'Art Gallery', 'French Restaurant', 'Exhibit',
       'Lebanese Restaurant', 'Cosmetics Shop', 'Bookstore',
       'Burger Joint', 'Art Museum', 'Sandwich Place',
       'Modern European Restaurant', 'Hotel', 'Ice Cream Shop',
       'Italian Restaurant', 'Seafood Restaurant', 'Dessert Shop',
       'Creperie', 'Beer Bar', 'Toy / Game Store', 'Tea Room',
       'Electronics Store', 'Church', 'Theater', 'Historic Site',
       'Bistro', 'Park', 'Comic Shop', 'Clothing Store',
       'Indie Movie Theater', 'Garden', 'Souvenir Shop', 'Pastry Shop',
       'Furniture / Home Store', 'Japanese Restaurant', 'Bakery',
       'Cheese Shop', 'Udon Restaurant', 'Gourmet Shop', 'Souvlaki Shop',
       'Szechuan Restaurant', 'Fountain', 'Wine Shop', 'Coffee Shop',
       'Gym / Fitness Center', 'Bar', 'Middle Eastern Restaurant',
       'Pizza Place', 'Thai Restaurant', 'Korean Restaurant',
       'Histor

In [88]:
# check if the results contain "Hotel"
"Hotel" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [89]:
# one hot encoding
Paris_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Paris_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Paris_onehot.columns[-1]] + list(Paris_onehot.columns[:-1])
Paris_onehot = Paris_onehot[fixed_columns]

print(Paris_onehot.shape)
Paris_onehot.head()

(2859, 207)


,Neighborhoods,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auvergne Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Basque Restaurant,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Bridge,Bubble Tea Shop,Burger Joint,Bus Stop,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Canal,Candy Store,Cemetery,Champagne Bar,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Doner Restaurant,Donut Shop,Electronics Store,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jiangxi Restaurant,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Library,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nightclub,Okonomiyaki Restaurant,Opera House,Organic Grocery,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Pop-Up Shop,Portuguese Restaurant,Provençal Restaurant,Pub,Radio Station,Record Shop,Recording Studio,Resort,Restaurant,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Snack Place,South American Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toy / Game Store,Track,Trail,Tram Station,Trattoria/Osteria,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Vineyard,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Historical quarters of Paris,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,Historical quarters of Paris,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Historical quarters of Paris,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [ ]:
# let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [90]:
Paris_grouped = Paris_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(Paris_grouped.shape)
Paris_grouped

(31, 207)


,Neighborhoods,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auvergne Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Basque Restaurant,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Bridge,Bubble Tea Shop,Burger Joint,Bus Stop,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Canal,Candy Store,Cemetery,Champagne Bar,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Doner Restaurant,Donut Shop,Electronics Store,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jiangxi Restaurant,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Library,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nightclub,Okonomiyaki Restaurant,Opera House,Organic Grocery,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Pop-Up Shop,Portuguese Restaurant,Provençal Restaurant,Pub,Radio Station,Record Shop,Recording Studio,Resort,Restaurant,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Snack Place,South American Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park Ride / Attraction,Toy / Game Store,Track,Trail,Tram Station,Trattoria/Osteria,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Vineyard,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Batignolles,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.000000,0.00,0.00,0.00,0.040000,0.040000,0.01,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.010000,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.02,0.000000,0.00,0.00,0.01,0.000000,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.220000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.01,0.00,0.00,0.000000,0.01,0.00,0.01,0.00,0.00,0.140000,0.01,0.01,0.00,0.01,0.00,0.00,0.070000,0.040000,0.00,0.00,0.01,0.00,0.01,0.00,0.01,0.01,0.00,0.010000,0.01,0.000000,0.00,0.01,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.010000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.010000,0.01,0.000000,0.000000,0.00,0.00,0.00,0.00,0.02,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,

In [91]:
len(Paris_grouped[Paris_grouped["Hotel"] > 0])

28

In [92]:
# Create a new DataFrame for Hotel data only

Paris_Hotel = Paris_grouped[["Neighborhoods","Hotel"]]

Paris_Hotel.head()

,Neighborhoods,Hotel
0,Batignolles,0.140000
1,"Belleville, Paris",0.000000
2,Bercy,0.040000
3,Cité des Fleurs,0.060000
4,Cour des miracles,0.074074


#### 7. Cluster Neighborhoods


In [93]:
# Run k-means to cluster the neighborhoods in Paris into 3 clusters

# set number of clusters

kclusters = 3

Paris_clustering = Paris_Hotel.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Paris_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 2, 2, 2, 0, 2, 1, 0], dtype=int32)

In [94]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
Paris_merged = Paris_Hotel.copy()

# add clustering labels
Paris_merged["Cluster Labels"] = kmeans.labels_

In [95]:
Paris_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
Paris_merged.head()

,Neighborhood,Hotel,Cluster Labels
0,Batignolles,0.140000,0
1,"Belleville, Paris",0.000000,1
2,Bercy,0.040000,1
3,Cité des Fleurs,0.060000,2
4,Cour des miracles,0.074074,2


In [96]:
# merge Paris_grouped with Paris_data to add latitude/longitude for each neighborhood
Paris_merged = Paris_merged.join(Paris_df.set_index("Neighborhood"), on="Neighborhood")

print(Paris_merged.shape)
Paris_merged.head() # check the last columns!

(31, 5)


,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
0,Batignolles,0.140000,0,48.883330,2.316670
1,"Belleville, Paris",0.000000,1,48.870180,2.384230
2,Bercy,0.040000,1,48.834880,2.384590
3,Cité des Fleurs,0.060000,2,48.892612,2.320326
4,Cour des miracles,0.074074,2,45.748650,4.881890


In [97]:
# sort the results by Cluster Labels
print(Paris_merged.shape)
Paris_merged.sort_values(["Cluster Labels"], inplace=True)
Paris_merged

(31, 5)


,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
0,Batignolles,0.140000,0,48.883330,2.316670
18,Paris Rive Gauche,0.100000,0,48.832060,2.339660
29,Trocadéro,0.160000,0,48.862340,2.288070
14,Les Halles,0.100000,0,48.863190,2.342010
12,"Javel, France",0.110000,0,48.843870,2.286130
9,Grenelle,0.110000,0,48.848420,2.291640
11,"Jardin de Tivoli, Paris",0.130000,0,48.862586,2.335726
6,Faubourg Saint-Germain,0.100000,0,48.857807,2.323826
25,Quartier des Grandes-Carrières,0.000000,1,43.760100,5.750995
8,Goutte d'Or,0.030000,1,48.885010,2.354130


In [98]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Paris_merged['Latitude'], Paris_merged['Longitude'], Paris_merged['Neighborhood'], Paris_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [99]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

#### 8. Examine Clusters

In [100]:
# Cluster 0

Paris_merged.loc[Paris_merged['Cluster Labels'] == 0]


,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
0,Batignolles,0.14,0,48.883330,2.316670
18,Paris Rive Gauche,0.10,0,48.832060,2.339660
29,Trocadéro,0.16,0,48.862340,2.288070
14,Les Halles,0.10,0,48.863190,2.342010
12,"Javel, France",0.11,0,48.843870,2.286130
9,Grenelle,0.11,0,48.848420,2.291640
11,"Jardin de Tivoli, Paris",0.13,0,48.862586,2.335726
6,Faubourg Saint-Germain,0.10,0,48.857807,2.323826


In [101]:
# Cluster 1

Paris_merged.loc[Paris_merged['Cluster Labels'] == 1]

,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
25,Quartier des Grandes-Carrières,0.00,1,43.760100,5.750995
8,Goutte d'Or,0.03,1,48.885010,2.354130
26,Revolutionary sections of Paris,0.02,1,48.857170,2.341400
10,Historical quarters of Paris,0.02,1,48.857170,2.341400
21,Quarters of Paris,0.02,1,48.857170,2.341400
27,Saint-Germain-des-Prés,0.02,1,48.853770,2.333310
13,"Latin Quarter, Paris",0.01,1,48.847750,2.351800
28,The Marais,0.02,1,48.858760,2.360870
2,Bercy,0.04,1,48.834880,2.384590
24,Quartier de La Chapelle,0.02,1,48.884150,2.357093


In [102]:
# Cluster 2

Paris_merged.loc[Paris_merged['Cluster Labels'] == 2]

,Neighborhood,Hotel,Cluster Labels,Latitude,Longitude
23,Quartier Pigalle,0.070000,2,48.882031,2.337576
15,Montmartre,0.060000,2,48.886150,2.337970
16,Montparnasse,0.080000,2,48.843130,2.321290
7,Front de Seine,0.080000,2,48.849310,2.285780
5,Faubourg Saint-Antoine,0.050000,2,48.850940,2.375670
4,Cour des miracles,0.074074,2,45.748650,4.881890
3,Cité des Fleurs,0.060000,2,48.892612,2.320326
20,Petit-Montrouge,0.090000,2,48.826420,2.325200
30,Épinettes,0.070000,2,48.842962,2.325298



######  A very small number of hotels are concentrated in the central area of Paris city.This represents a great opportunity and high potential areas to open new hotel as there is very little to no competition from existing hotels.

